In [25]:
import pickle
import pandas as pd

In [7]:
traindata=pickle.load(open('EDA_Vehicledata.pkl','rb'))

In [8]:
traindata

,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage_Yes,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender,Vehicle_Age_1_2_Years,Vehicle_Age_lt_1_Year,Vehicle_Age_gt_2_Years
0,44,1,28.0,0,0,40454.0,26.0,217,1,1,0,0,1
1,76,1,3.0,0,1,33536.0,26.0,183,0,1,1,0,0
2,47,1,28.0,0,0,38294.0,26.0,27,1,1,0,0,1
3,21,1,11.0,1,1,28619.0,152.0,203,0,1,0,1,0
4,29,1,41.0,1,1,27496.0,152.0,39,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381104,74,1,26.0,1,1,30170.0,26.0,88,0,1,1,0,0
381105,30,1,37.0,1,1,40016.0,152.0,131,0,1,0,1,0
381106,21,1,30.0,1,1,35118.0,160.0,161,0,1,0,1,0
381107,68,1,14.0,0,0,44617.0,124.0,74,0,0,0,0,1


Now let's build a pipeline for preprocessing the categorical features/attributes:

In [9]:
# Numerical and categorical columns
num_df= ['Age','Vintage']
cat_df = ['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age_lt_1_Year',
            'Vehicle_Age_gt_2_Years','Vehicle_Damage','Region_Code','Policy_Sales_Channel']

In [10]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer



cat_pipeline = Pipeline([
        # we will fill the NaNs with the mode
        ('imputer', SimpleImputer(strategy="most_frequent")),
        # the features has order meaning
        ('encoder', OrdinalEncoder()),
    ])
X=traindata.drop(['Response'], axis = 1)

# prepare the df form the ML models by calling the full_pipeline
X_prepared = cat_pipeline.fit_transform(X)
# inspect the number of rows & columns of the prepared df
X_prepared.shape

(381109, 12)

Now let's build a pipeline for preprocessing all the attributes:

In [11]:
traindata.columns

Index(['Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Vehicle_Damage_Yes', 'Annual_Premium', 'Policy_Sales_Channel',
       'Vintage', 'Response', 'Gender', 'Vehicle_Age_1_2_Years',
       'Vehicle_Age_lt_1_Year', 'Vehicle_Age_gt_2_Years'],
      dtype='object')

In [12]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OrdinalEncoder())])

In [14]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
ss = StandardScaler()
traindata[num_df] = ss.fit_transform(traindata[num_df])


mm = MinMaxScaler()
traindata[['Annual_Premium']] = mm.fit_transform(traindata[['Annual_Premium']])

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from scipy.stats import randint
import pickle
# import xgboost as xgb
# import lightgbm as lgb
from sklearn.metrics import accuracy_score
# import packages for hyperparameters tuning
# from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, KFold, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score,accuracy_score,confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, classification_report

In [16]:
from sklearn.model_selection import train_test_split

train_target=traindata['Response']
train=traindata.drop(['Response'], axis = 1)
x_train,x_test,y_train,y_test = train_test_split(train,train_target, test_size=0.3, random_state = 0)

In [17]:
x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape

((266776, 12), (114333, 12), (266776,), (114333,))

# DecisionTreeClassifier

In [18]:
from sklearn.tree import DecisionTreeClassifier

dtc=DecisionTreeClassifier()
dtc.fit(x_train,y_train)

DecisionTreeClassifier()

In [19]:
y_pred=dtc.predict(x_test)

In [22]:
pickle.dump(dtc, open('dtc_model.pkl', 'wb'))

In [23]:
from sklearn.metrics import accuracy_score,classification_report

print("Accuracy Score: ",accuracy_score(y_pred,y_test))
print("\n")
print(classification_report(y_pred,y_test))

Accuracy Score:  0.8233755783544559


              precision    recall  f1-score   support

           0       0.90      0.90      0.90     99393
           1       0.31      0.30      0.30     14940

    accuracy                           0.82    114333
   macro avg       0.60      0.60      0.60    114333
weighted avg       0.82      0.82      0.82    114333



In [26]:
pd.DataFrame(confusion_matrix(y_test,y_pred),columns=['Predicted No','Predicted Yes'],index=['Actual No','Actual Yes'])


,Predicted No,Predicted Yes
Actual No,89720,10521
Actual Yes,9673,4419


# Saving the Dtc model to pickle file

In [27]:
with open('dtc_model.pkl', 'wb') as f:
    pickle.dump(dtc, f)

# Checking the acuracy with different models

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [371]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=3),
    "                         Decision Tree": DecisionTreeClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "                        Neural Network": MLPClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier()
                                   
}

for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    print(name + " : {:.2f}%".format(accuracy_score(y_pred,y_test)*100))

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                   Logistic Regression : 87.67%
                   K-Nearest Neighbors : 84.08%
                         Decision Tree : 82.32%
                         Random Forest : 86.58%


D:\Anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Support Vector Machine (Linear Kernel) : 87.67%


D:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network : 87.64%
                     Gradient Boosting : 87.67%


# So the more best model is from Logistic Regression and Random Forest Classifier as their accuracy is more.
Lets build the model using Random Forest Classifier.

In [29]:
model=RandomForestClassifier()
model.fit(x_train,y_train)

RandomForestClassifier()

In [30]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,confusion_matrix,r2_score,accuracy_score
y_pred=model.predict(x_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n" , pd.DataFrame(confusion_matrix(y_test,y_pred),columns=['Predicted No','Predicted Yes'],index=['Actual No','Actual Yes']))
print("\nTraining Score: ",model.score(x_test,y_test)*100)
print("\n Mean Squared Error: ",mean_squared_error(y_test,y_pred))
print("\n R2 score is: ",r2_score(y_test,y_pred), "\n")
print(model.get_params())
print("\nAccuracy Score:  ", accuracy_score(y_test,y_pred)*100)



Classification Report is:
               precision    recall  f1-score   support

           0       0.89      0.97      0.93    100241
           1       0.36      0.12      0.18     14092

    accuracy                           0.87    114333
   macro avg       0.62      0.54      0.55    114333
weighted avg       0.82      0.87      0.83    114333

Confusion Matrix:
             Predicted No  Predicted Yes
Actual No          97250           2991
Actual Yes         12404           1688

Training Score:  86.5349461660238

 Mean Squared Error:  0.13465053833976193

 R2 score is:  -0.24604368176073477 

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'ver

# Saving the  RandomForest Model

In [31]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)